In [1]:
import pkg_resources
import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
required = {'dash', 'dash-core-components', 'dash-html-components', 'dash-daq', 'cvxopt' }
missing = required - installedPackages
if missing:
    !pip install dash==1.9.1
    !pip install dash-core-components==1.8.1
    !pip install dash-html-components==1.0.2
    !pip install dash-daq==0.4.0
    !pip install cvxopt==1.2.5

C:\Users\payal\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


  Created wheel for dash: filename=dash-1.9.1-py3-none-any.whl size=71526 sha256=98756588ccc8f36f74fb04edb069423e2e89ced8120a69d65eb4264239df3768
  Stored in directory: c:\users\payal\appdata\local\pip\cache\wheels\aa\0c\f3\67e9780125e2e642e423bbe462d32c07be9a335c57de37f7da
  Created wheel for dash-core-components: filename=dash_core_components-1.8.1-py3-none-any.whl size=3395715 sha256=58fad65e26008715b6c2f524fa8ed10eb3439ccf63b236cb8ef465d7bb29f5eb
  Stored in directory: c:\users\payal\appdata\local\pip\cache\wheels\f2\f2\d9\6ecefa889bd85d93bb61ddd1ff31244c9a0203a44122d5e0c0
  Created wheel for dash-html-components: filename=dash_html_components-1.0.2-py3-none-any.whl size=427919 sha256=94dd5ee83563c08c7890b0de0b0509d7d16ccc4122dec0615a8ef0ae4ae19864
  Stored in directory: c:\users\payal\appdata\local\pip\cache\wheels\a5\fd\59\de9c505e7fb39d83141c560ce9fdf6ce52cb7f8d2238e387df
  Created wheel for dash-renderer: filename=dash_renderer-1.2.4-py3-none-any.whl size=1146877 sha256=311b233

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\payal\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\payal\\AppData\\Local\\Temp\\pip-install-o1nj3a88\\cvxopt_0d0697ca5a5f4a7285eb435ede0870d7\\setup.py'"'"'; __file__='"'"'C:\\Users\\payal\\AppData\\Local\\Temp\\pip-install-o1nj3a88\\cvxopt_0d0697ca5a5f4a7285eb435ede0870d7\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\payal\AppData\Local\Temp\pip-wheel-4a79x2yi'
       cwd: C:\Users\payal\AppData\Local\Temp\pip-install-o1nj3a88\cvxopt_0d0697ca5a5f4a7285eb435ede0870d7\
  Complete output (23 lines):
  C:\Users\payal\anaconda3\lib\site-packages\setuptools\dist.py:757: UserWarning: Usage of dash-separated 'descriptio

In [211]:
def predict_riskTolerance(X_input):

    filename = 'finalized_model.sav'
    loaded_model = load(open(filename, 'rb'))
    # estimate accuracy on validation set
    predictions = loaded_model.predict(X_input)
    return predictions

#Asset allocation given the Return, variance
def get_asset_allocation(riskTolerance,stock_ticker):
    #ipdb.set_trace()
    assets_selected = assets.loc[:,stock_ticker]
    return_vec = np.array(assets_selected.pct_change().dropna(axis=0)).T
    n = len(return_vec)
    returns = np.asmatrix(return_vec)
    mus = 1-riskTolerance
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(return_vec))
    pbar = opt.matrix(np.mean(return_vec, axis=1))
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    # Calculate efficient frontier weights using quadratic programming
    portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    w=portfolios['x'].T
    print (w)
    Alloc =  pd.DataFrame(data = np.array(portfolios['x']),index = assets_selected.columns)

    # Calculate efficient frontier weights using quadratic programming
    portfolios = solvers.qp(mus*S, -pbar, G, h, A, b)
    returns_final=(np.array(assets_selected) * np.array(w))
    returns_sum = np.sum(returns_final,axis =1)
    returns_sum_pd = pd.DataFrame(returns_sum, index = assets.index )
    returns_sum_pd = returns_sum_pd - returns_sum_pd.iloc[0,:] + 100   
    return Alloc,returns_sum_pd



#Callback for the graph
#This function takes all the inputs and computes the cluster and the risk tolerance




def update_risk_tolerance(n_clicks,Age,Nwcat,Inccl,Risk,Edu,Married,Kids,Occ):
    RiskTolerance = 0
    if n_clicks != None:    
        X_input = [[Age,Edu,Married,Kids,Occ,Inccl, Risk,Nwcat]]
        RiskTolerance= predict_riskTolerance(X_input)
    #print(RiskAversion)
    #Using linear regression to get the risk tolerance within the cluster.    
    return list([round(float(RiskTolerance*100),2)])

@app.callback([Output('Asset-Allocation', 'figure'),
              Output('Performance', 'figure')],
            [Input('submit-asset_alloc_button', 'n_clicks'),
            Input('risk-tolerance-text', 'value')], 
            [State('ticker_symbol', 'value')
            ])
def update_asset_allocationChart(n_clicks, risk_tolerance, stock_ticker):
    
    Allocated, InvestmentReturn = get_asset_allocation(risk_tolerance,stock_ticker)  
    
    return [{'data' : [go.Bar(
                        x=Allocated.index,
                        y=Allocated.iloc[:,0],
                        marker=dict(color='red'),
                    ),
                    ],
            'layout': {'title':" Asset allocation - Mean-Variance Allocation"}

       },
            {'data' : [go.Scatter(
                        x=InvestmentReturn.index,
                        y=InvestmentReturn.iloc[:,0],
                        name = 'OEE (%)',
                        marker=dict(color='red'),
                    ),
                    ],
            'layout': {'title':"Portfolio value of $100 investment"}

       }]

app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output,State
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import dash_daq as daq
from pickle import load
import cvxopt as opt
from cvxopt import blas, solvers

In [284]:
import os
os.getcwd()
from gettext import install
import pandas_datareader.data as web 
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime as dt
from scipy.optimize import minimize 
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)
from scipy.optimize import Bounds
pd.set_option('display.max_columns', None)
import plotly
import seaborn as sns
import plotly.express as px

In [285]:
#!pip install PyPortfolioOpt

from pypfopt import expected_returns
from pypfopt import risk_models
import plotly.offline as py
import plotly.figure_factory as ff

In [371]:
import dash_table as dte

In [381]:
def portfolio_recommender(Risk_profile):
    
    def download_etf(symbol,source,start_date='2012-00-01',end_date='2022-10-01'):
    
        data = web.DataReader(symbol,source,start=start_date,end=end_date) 
        data = data.sort_index() 

        return data 

    def download_etfs(symbols,source,start_date='2012-01-01',end_date='2022-10-01'):  

    
        dfs = [] # empty list of pandas dataframes 

        for sym in symbols: 
            df = download_etf(sym,source,start_date=start_date,end_date=end_date)
            tmpdf = df['Close']
            dfs.append(tmpdf)  

        df = pd.concat(dfs,axis=1) 
        df.columns = symbols 

        return df 
    
    symbols = ["VCSH","AGG","VTV","SPYV","VGLT","SPY","SPLG","ZROZ"]
    
    def plot_table(table, index_title = "", title = "" , layout_width = 900):
        colorscale = [[0, '#4d004c'],[.5, '#f2e5ff'],[1, '#ffffff']]
        table = ff.create_table(round(table,4), index = True, index_title=index_title, colorscale=colorscale )
        table.update_layout(title_text = title)
        table.layout.width=layout_width
        py.iplot(table)


    asset_select = download_etfs(symbols, "stooq","2012-01-01","2022-10-31")
    asset_select = asset_select.dropna()
    #Annualized Returns calculation 
    returns = expected_returns.mean_historical_return(asset_select)
    
    #Variance of Portfolio calculation 
    variance = risk_models.sample_cov(asset_select)
    
     
    
    sector_mapper = {"VCSH": "Corporate Bond","VCIT": "Corporate Bond","LQD": "Corporate Bond","HYG": "Corporate Bond",
                 "VCLT": "Corporate Bond","USIG":"Corporate Bond","SPBO":"Corporate Bond","SPIB":"Corporate Bond",
                 "IGIB":"Corporate Bond","MORT":"Real estate","PSR":"Real estate","VNQI":"Real estate","KBWY":"Real estate",
                   "SPTL":"Treasury bonds","VGLT":"Treasury bonds","TLT":"Treasury bonds","AGG":"Treasury bonds",
                 "BND":"Treasury bonds","EDV":"Treasury bonds","ZROZ":"Treasury bonds","TLH":"Treasury bonds",
                 "SPY": "Equity","IVV":"Equity","VOO":"Equity","VTI":"Equity","VTV":"Equity","RSP":"Equity",
                 "IJR":"Equity","SPLG":"Equity","SPYV":"Equity"
                }
    
    ## Conservative Portfolio 
    csector_upper = {"Corporate Bond":0.60, "Real estate": 0.15, "Treasury bonds":0.6, "Equity":0.2}
    csector_lower = {"Corporate Bond": 0.10, "Equity":0.1}
    #Optimizing the portfolio using the above equation. 
    from pypfopt.efficient_frontier import EfficientFrontier
    ef = EfficientFrontier(returns, variance , weight_bounds=(0, 1))
    ef.add_sector_constraints(sector_mapper, csector_lower, csector_upper)
    #ef.efficient_risk(0.05, market_neutral=False)
    ef.min_volatility()



    #Getting the optimal weights from the optimization
    weights = ef.clean_weights()

    #Add weights to the weighs dataframe
    weights_df = pd.DataFrame(list(weights.values()), index=list(weights.keys()), columns = ["Optimal weights"])
    
    
    plot_table(weights_df,layout_width = 300, index_title = "Tickers" )

    cportfolio_performance = pd.DataFrame(ef.portfolio_performance(risk_free_rate=0), 
                                     index = ["Annual return", "Annual volatility", "Sharpe Ratio"],
                                     columns = ["Mean_Variance"])
    cportfolio = plot_table(cportfolio_performance,layout_width = 300, index_title = "PERFORMANCE")
    
    #if Risk_profile == 'Conservative':
    port = cportfolio_performance
    
    return port

In [382]:
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [383]:
app

In [384]:
app.layout = html.Div([
    html.Div([ 
        #Dashboard Name
        html.Div([
            html.H3(children='Robo Advisor Dashboard'),
            html.Div([
                html.H5(children='Step 1 : Enter information to assess risk profile'),            
                ],style={'display': 'inline-block','vertical-align': 'top',  'width': '30%',\
                         'color':'White', 'background-color': 'Black'}), 
            html.Div([
                html.H5(children='Step 2 : Asset Allocation and Portfolio performance'),            
                ],style={'display': 'inline-block', 'vertical-align': 'top',  \
                         'color':'white','horizontalAlign' : "left", 'width': '70%', 'background-color':'black'}), 
            ],style={'font-family': 'calibri'}),        
         
         #All the Investor Characteristics
                      
       #********************Demographics Features DropDown********
         html.Div([   
          html.Div([ 
            
            html.Label('Age:',style={'padding': 5}),
            dcc.Slider(
                id='Age', 
                min = 0, max = 60, step = 10,
                marks={0: '0',10: '10', 20: '20', 30:'30',40:'40', 50:'50', 60: '>60' },                
                value=0),
            #html.Br(),
            
            html.Label('Please enter your average income in the last 3 years:', style={'padding': 5}),
            dcc.Slider(
                id='Nwcat', 
                min = 0, max = 200000, step = 50000,
                marks={0: '0',100000: '100000',150000: '150000',200000: '> 150000',},                
                value=0),
            #html.Br(),
              
            html.Label('Please enter capital amount wanting to invest', style={'padding': 5}),
            dcc.Slider(
                id='Inccl',
                #min = investors['INCOME07'].min(), max = investors['INCOME07'].max(),
                min = 0,
                max = 150000,
                marks={0: '0',20000: '$20K',100000: '$100K',150000: '>100k',},
                value=0),
              
            html.Label('Please enter your investment horizon', style={'padding': 5}),
            dcc.Slider(
                id='horizon',
                min = 0,
                max = 20,
                step = 1,
                marks={0: '0',5: '5',10: '10',15: '>15',},
                value=0),
              
            # html.Br(),
            html.Label('What is your understanding of stocks, bonds, and ETFs?', style={'padding': 5}),
            dcc.Dropdown(
                        id='Edu',
                        options = [
                                    {'label': 'None', 'value': 'None'},
                                    {'label': 'Some', 'value': 'Some'},
                                    {'label': 'Good', 'value': 'Good'},
                                    {'label': 'Extensive', 'value': 'Extensive'},
                        ],
                        #value = ['None','Some','Good','Extensive'],
                        placeholder="Select an option",
                        multi = False, 
                        # style={'fontSize': 24, 'width': 75}
                        ),
              
            html.Label('When you hear “risk” related to your finances, what is the first thought that comes to mind?', style={'padding': 5}),
            dcc.Dropdown(
                        id='risk',
                        options = [
                                    {'label': 'worry', 'value': 'worry'},
                                    {'label': 'patience', 'value': 'patience'},
                                    {'label': 'opportunity', 'value': 'opportunity'},
                                    {'label': 'thrill', 'value': 'thrill'},
                        ],
                        #value = ['None','Some','Good','Extensive'],
                        placeholder="Select an option",
                        multi = False, 
                        # style={'fontSize': 24, 'width': 75}
                        ),
              
            html.Label('From 1 to 5, how comfortable Would you feel if your profolio looses 20% of value in one day ? \n 1 = not comfortable at all  5 = very comfortable:', style={'padding': 5}),
            dcc.Slider(
                id='Risk',
                min = 1,
                max = 5,
                marks={ 1: '1',2: '2',3: '3',4: '4', 5: '5'},
                value=1),
              
            html.Label('How would you describe your approach to making important financial decisions?', style={'padding': 5}),
            dcc.Dropdown(
                        id='aversion',
                        options = [
                                    {'label': 'I try to avoid making decisions', 'value': 'I try to avoid making decisions'},
                                    {'label': 'I reluctantly make decisions', 'value': 'I reluctantly make decisions'},
                                    {'label': 'I confidently make decisions', 'value': 'I confidently make decisions'},
                        ],
                        #value = ['None','Some','Good','Extensive'],
                        placeholder="Select an option",
                        multi = False, 
                        # style={'fontSize': 24, 'width': 75}
                        ),
              
             
            #html.Br(),
            html.Button(id='investor_char_button',
                            n_clicks = 0,
                            children = 'Calculate Risk Profile',
                            style = {'fontSize': 14, 'marginLeft': '30px', 'color' : 'white',\
                                     'horizontal-align': 'left','backgroundColor': 'grey'}),             
            #html.Br(),            
              ],style={'width': '80%'}),           
            
            ],style={'width': '30%', 'font-family': 'calibri','vertical-align': 'top','display': 'inline-block'\
                     }),
#                     , "border":".5px black solid"}),

    # ********************Risk Tolerance Charts********            
       html.Div([    
               #html.H5(children='Step 2 : Enter the Instruments for the allocation portfolio'),  
        html.Div([
            html.Div([ 
                html.Label('Risk Profile:', style={'padding': 5}),
                dcc.Input(id= 'risk-tolerance-text'),
               
                ],style={'width': '100%','font-family': 'calibri','vertical-align': 'top','display': 'inline-block'}),
            
            
            
        html.Div([ 
                html.Label('Select the assets for the portfolio:', style={'padding': 5}),
                #dcc.Dropdown(
                        #id='ticker_symbol',
                        #options = options,
                        #value = ['GOOGL', 'FB', 'GS','MS','GE','MSFT'], 
                        #multi = True
                        # style={'fontSize': 24, 'width': 75}
                       # ),
                html.Button(id='submit-asset_alloc_button',
                            n_clicks = 0,
                            children = 'Submit',
                            style = {'fontSize': 12, 'marginLeft': '25px','color' : 'white', 'backgroundColor': 'grey'}

                ), 
               ],style={'width': '100%','font-family': 'calibri','vertical-align': 'top','display': 'inline-block'}),
            ],style={'width': '100%','display': 'inline-block','font-family': 'calibri','vertical-align': 'top'}),
           
           html.Div([                
                html.Div([
                    dcc.Graph(id='Asset-Allocation'), 
                    ], style={'width': '50%', 'vertical-align': 'top', 'display': 'inline-block', \
                      'font-family': 'calibri', 'horizontal-align': 'right'}),
                html.Div([
                    dcc.Graph(id='Performance')
                    ], style={'width': '50%', 'vertical-align': 'top', 'display': 'inline-block', \
                      'font-family': 'calibri', 'horizontal-align': 'right'}),
                   ], style={'width': '100%', 'vertical-align': 'top', 'display': 'inline-block', \
                          'font-family': 'calibri', 'horizontal-align': 'right'}),          


        ], style={'width': '70%','display': 'inline-block','font-family': 'calibri','vertical-align': 'top', 'horizontal-align': 'right'}),
       ],style={'width': '70%','display': 'inline-block','font-family': 'calibri','vertical-align': 'top'}),  
    
])   

In [385]:
def riskprofile(Age,Nwcat,Inccl,horizon,Edu,risk, Risk,aversion):
    age = float(Age)
    if age > 50:
        age_score = 0
    elif age <=50 and age > 30:
        age_score = 10
    else:
        age_score = 15
        
    Nwcat = float(Nwcat)
    if Nwcat > 150000:
        income_score = 15
    elif Nwcat > 100000 and Nwcat <= 150000:
        income_score = 10
    else:
        income_score = 0
        
        
    Inccl = float(Inccl)
    if Inccl > 100000:
        capital_score = 15
    elif Inccl > 20000 and Inccl <= 100000:
        capital_score = 10 
    else:
        capital_score = 0
        
        
    horizon = int(horizon)
    if horizon > 10:
        inv_hor_score = 15
    elif horizon > 10 and horizon <=5:
        inv_hor_score = 10 
    else: 
        inv_hor_score = 0
        
        
    Edu = Edu
    if Edu == "None":
        inv_knowledge_nmb = 0
    elif Edu == "Some":
        inv_knowledge_nmb = 7
    elif Edu == "Good":
        inv_knowledge_nmb = 11
    else:
        inv_knowledge_nmb = 15
        
        
    risk = risk
    if risk == "worry":
        risk_perc_nmb = 0
    elif risk == "patience":
        risk_perc_nmb = 5
    elif risk == "opportunity":
        risk_perc_nmb = 10
    else:
        risk_perc_nmb = 15 
        
        
    aversion = aversion
    if aversion == "I try to avoid making decisions":
        regret_av_nmb = 0
    elif aversion == "I reluctantly make decisions":
        regret_av_nmb = 6
    else:
        regret_av_nmb = 15
        
        
    Risk_score = inv_knowledge_nmb + risk_perc_nmb + 3* Risk + regret_av_nmb + age_score +income_score +  capital_score + inv_hor_score
    
    if Risk_score <= 24 and Risk_score >0:
        Risk_profile = "Conservative"
    elif Risk_score <= 48 and Risk_score >=24:
        Risk_profile = "Moderately Conservative"
    elif Risk_score <= 72 and Risk_score >=48:
        Risk_profile = "Moderate"
    elif Risk_score <= 96 and Risk_score >=72:
        Risk_profile = "Moderately Aggressive"
    else:
        Risk_profile = "Aggressive"
        
    return Risk_profile
        
    

In [386]:
@app.callback(
     Output('risk-tolerance-text', 'value'),
    [Input('investor_char_button', 'n_clicks'),
    Input('Age', 'value'),Input('Nwcat', 'value'),
    Input('Inccl', 'value'), Input('horizon', 'value'),
    Input('Edu', 'value'),Input('risk', 'value'),
    Input('Risk', 'value'), Input('aversion', 'value')])
#get the x and y axis details 

def update_risk_tolerance(n_clicks, Age,Nwcat,Inccl,horizon,Edu,risk, Risk,aversion):
    RiskTolerance = "None"
    if n_clicks != None:    
        X_input = [Age,Nwcat,Inccl,horizon,Edu,risk, Risk,aversion]
        RiskTolerance= riskprofile(Age,Nwcat,Inccl,horizon,Edu,risk, Risk,aversion)#(X_input)
    #print(RiskAversion)
    #Using linear regression to get the risk tolerance within the cluster.    
    return RiskTolerance



@app.callback([Output('Asset-Allocation', 'figure'),
              Output('Performance', 'figure')],
            [Input('submit-asset_alloc_button', 'n_clicks'),
            Input('risk-tolerance-text', 'value')])

def update_asset_allocationChart(n_clicks, risk_tolerance):
    
    Allocated = portfolio_recommender(risk_tolerance)  
    
    return [{'data' : Allocated,
            'layout': {'title':" Asset allocation"}

       },
            {'data' : Allocated,
            'layout': {'title':"Portfolio value of $100 investment"}

       }]
        
        

In [ ]:
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Dec/2022 09:29:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2022 09:29:06] "GET /_dash-component-suites/dash_table/bundle.v4_6_1m1671036866.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2022 09:29:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2022 09:29:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2022 09:29:06] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\payal\anaconda3\lib\site-packages\dash\dash.py", line 1384, in add_context
    jsonResponse = json.dumps(
  File "C:\Users\payal\anaconda3\lib\json\__init__.py", line 234, in dumps
    return cls(
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 59, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 136, in default
    return _json.JSONEncoder.default(self, obj)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type DataFrame is no

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\payal\anaconda3\lib\site-packages\dash\dash.py", line 1384, in add_context
    jsonResponse = json.dumps(
  File "C:\Users\payal\anaconda3\lib\json\__init__.py", line 234, in dumps
    return cls(
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 59, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 136, in default
    return _json.JSONEncoder.default(self, obj)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type DataFrame is no

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\payal\anaconda3\lib\site-packages\dash\dash.py", line 1384, in add_context
    jsonResponse = json.dumps(
  File "C:\Users\payal\anaconda3\lib\json\__init__.py", line 234, in dumps
    return cls(
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 59, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 136, in default
    return _json.JSONEncoder.default(self, obj)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type DataFrame is no

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\payal\anaconda3\lib\site-packages\dash\dash.py", line 1384, in add_context
    jsonResponse = json.dumps(
  File "C:\Users\payal\anaconda3\lib\json\__init__.py", line 234, in dumps
    return cls(
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 59, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 136, in default
    return _json.JSONEncoder.default(self, obj)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type DataFrame is no

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\payal\anaconda3\lib\site-packages\dash\dash.py", line 1384, in add_context
    jsonResponse = json.dumps(
  File "C:\Users\payal\anaconda3\lib\json\__init__.py", line 234, in dumps
    return cls(
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 59, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 136, in default
    return _json.JSONEncoder.default(self, obj)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type DataFrame is no

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\payal\anaconda3\lib\site-packages\dash\dash.py", line 1384, in add_context
    jsonResponse = json.dumps(
  File "C:\Users\payal\anaconda3\lib\json\__init__.py", line 234, in dumps
    return cls(
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 59, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 136, in default
    return _json.JSONEncoder.default(self, obj)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type DataFrame is no

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\payal\anaconda3\lib\site-packages\dash\dash.py", line 1384, in add_context
    jsonResponse = json.dumps(
  File "C:\Users\payal\anaconda3\lib\json\__init__.py", line 234, in dumps
    return cls(
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 59, in encode
    encoded_o = super(PlotlyJSONEncoder, self).encode(o)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "C:\Users\payal\anaconda3\lib\site-packages\_plotly_utils\utils.py", line 136, in default
    return _json.JSONEncoder.default(self, obj)
  File "C:\Users\payal\anaconda3\lib\json\encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type DataFrame is no

127.0.0.1 - - [15/Dec/2022 09:54:41] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
def update_asset_allocationChart(Risk_profile):
    
    Allocated = portfolio_recommender(Risk_profile)  
    
    return cportfolio,
            'layout': {'title':" Asset allocation - Mean-Variance Allocation"}

       